In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install control

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 11.7 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sympy as sp
import control

In [5]:
x1,x2,x3,x4,x5,x6,m,g,ux,uy=sp.symbols("x1 x2 x3 x4 x5 x6 m g ux uy")

In [6]:
x=sp.Matrix([x1,x2,x3,x4,x5,x6])

In [7]:
m=1
g=10

In [42]:
f1=x2
f2=(ux*(sp.cos(x5))-uy*(sp.sin(x5)))/m
f3=x4
f4=(ux*(sp.sin(x5))+uy*(sp.cos(x5))-g)/m
f5=x6
f6=0

In [43]:
f=sp.Matrix([f1,f2,f3,f4,f5,f6])

In [44]:
A=f.jacobian(x)
B=f.jacobian([ux,uy])

In [45]:
A

Matrix([
[0, 1, 0, 0,                        0, 0],
[0, 0, 0, 0, -ux*sin(x5) - uy*cos(x5), 0],
[0, 0, 0, 1,                        0, 0],
[0, 0, 0, 0,  ux*cos(x5) - uy*sin(x5), 0],
[0, 0, 0, 0,                        0, 1],
[0, 0, 0, 0,                        0, 0]])

In [46]:
B

Matrix([
[      0,        0],
[cos(x5), -sin(x5)],
[      0,        0],
[sin(x5),  cos(x5)],
[      0,        0],
[      0,        0]])

In [47]:
subs={x1:0,x2:0,x3:0,x4:-1,x5:0,x6:0,ux:0,uy:0}
A_lin = A.subs(subs)
B_lin = B.subs(subs)

In [48]:
A_lin.eigenvals()

{0: 6}

In [49]:
B_lin

Matrix([
[0, 0],
[1, 0],
[0, 0],
[0, 1],
[0, 0],
[0, 0]])

In [50]:
Q=np.diag([0,10,0,100,10,10])
R=np.diag([1,1])


In [51]:
obs=np.array([0,0,1.2,0,0,0])


In [52]:
obs

array([0. , 0. , 1.2, 0. , 0. , 0. ])

In [4]:
import gym
import matplotlib.pyplot as plt
from gym.wrappers import RecordVideo
import math

In [195]:
def controller(obs):
    kp_x=1
    kd_x=2.5
    kp_vy=2
    kp_angle=2.5
    kd_angle=2.5

    angle_todo =  ( obs[4])*kp_angle + (obs[5])*kd_angle - kp_x*obs[0] -kd_x*obs[1]
    uy=kp_vy*(-0.1-obs[3])
    if(obs[6] or obs[7]):
        angle_todo=0
        uy=0
    u=[angle_todo,uy]
    print(u)
    return u

In [ ]:
env = RecordVideo(gym.make("LunarLanderContinuous-v2"), "./mp4")
# env=RecordVideo(CustomCartPoleEnv(),"./mp4")
env.env.seed(1)     # seed for reproducibility
obs = env.env.reset()
x_vel_error=0
integral=0
for i in range(1000):
    env.render()
    # obs=[float(obs[0]/2.5),float(obs[2]/10),float(obs[1]/2.5),float(obs[3]/10),float(obs[4]/6.2831855) ,float(obs[5]/10)]
    
    obs=[obs[0],obs[2],obs[1],obs[3],obs[4],obs[5],obs[6],obs[7]]
    # x,xdot,y,ydot,theta,thetadot
    val=controller(obs)
    vx=val[0]
    vy=val[1]
    forcey = (float(np.clip(vy, 0, 1)))
    forcex= (float(np.clip(vx, -1, 1)))
    action=np.array([forcey,forcex])
    obs,_,done,_,=env.step(action)
    # print('\n')
    if done:
        print(f'Terminated after {i+1} iterations.')
        break

env.close()


In [123]:
pip install wheel setuptools pip --upgrade

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install swig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install gymnasium[box2d]

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'box2d-py' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'box2d-py'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379444 sha256=93bf510cf69d1f6a49b2e84a4b3186a935d7f2f7da76eaf5bc6c3ec0fc5f5c9e
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py
Note: you may need to restart the kernel to use updated packages.
